In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

# 单纯形法

In [15]:
# 定义线性规划求解函数
def lp_solve(matrix: pd.DataFrame):    # 表示输入的数据格式为DataFrame
    """
    输入线性规划的矩阵，根据单纯形法求解线性规划模型
     max cx
    s.t. ax<=b
    矩阵形式是：
             b    x1    x2   x3   x4   x5
    obj    0.0  70.0  30.0  0.0  0.0  0.0
    x3   540.0   3.0   9.0  1.0  0.0  0.0
    x4   450.0   5.0   5.0  0.0  1.0  0.0
    x5   720.0   9.0   3.0  0.0  0.0  1.0
    第1行是目标函数的系数
    第2-4行是约束方程的系数
    第1列是约束方程的常数项
    obj-b 交叉，即第1行第1列的元素是目标函数的负值
    x3,x4,x5 是松弛变量，也是初始可行解
    :param matrix:
    :return:
    """
    
    # 检验数是否大于0
    c = matrix.iloc[0, 1:]
    while c.max() > 0:
        
        # 1. 选择入基变量, 即目标函数系数最大的变量入基
        c = matrix.iloc[0, 1:]
        in_x = c.idxmax()
        in_x_v = c[in_x] # 入基变量的系数
        
        # 2. 选择出基变量
        # 选择正的最小比值对应的变量出基min (b列/入基变量)
        b = matrix.iloc[1:, 0]
        in_x_a = matrix.iloc[1:][in_x]  # 选择入基变量对应的列
        out_x = (b / in_x_a).idxmin()  # 得出出基变量idx
        
        # 3. 旋转操作
        matrix.loc[out_x, :] = matrix.loc[out_x, :] / matrix.loc[out_x, in_x]
        for idx in matrix.index:
            if idx != out_x:
                matrix.loc[idx, :] = matrix.loc[idx, :] - matrix.loc[out_x, :] * matrix.loc[idx, in_x]
        
        # 4. 基索引替换(入基与出基变量名称替换)
        index = matrix.index.tolist()
        i = index.index(out_x) # 下标
        index[i] = in_x
        matrix.index = index
        print(matrix)
    # 打印结果
    print("最终的最优单纯形法是:")
    print(matrix)
    print("目标函数值是:", - matrix.iloc[0, 0])
    print("最优决策变量是:")
    x_count = (matrix.shape[1] - 1) - (matrix.shape[0] - 1)
    X = matrix.iloc[0, 1:].index.tolist()[: x_count]
    for xi in X:
        print(xi, '=', matrix.loc[xi, 'b'])


In [16]:
# 主函数
def main():
    # 增广矩阵
    matrix = pd.DataFrame(
        np.array([
            [0, 70, 30, 0, 0, 0],
            [540, 3, 9, 1, 0, 0],
            [450, 5, 5, 0, 1, 0],
            [720, 9, 3, 0, 0, 1]]),
        index = ['obj', 'x3', 'x4', 'x5'],
        columns = ['b', 'x1', 'x2', 'x3', 'x4', 'x5']
    )
    lp_solve(matrix)

In [17]:
main()

          b   x1        x2   x3   x4        x5
obj -5600.0  0.0  6.666667  0.0  0.0 -7.777778
x3    300.0  0.0  8.000000  1.0  0.0 -0.333333
x4     50.0  0.0  3.333333  0.0  1.0 -0.555556
x1     80.0  1.0  0.333333  0.0  0.0  0.111111
          b   x1   x2   x3   x4        x5
obj -5700.0  0.0  0.0  0.0 -2.0 -6.666667
x3    180.0  0.0  0.0  1.0 -2.4  1.000000
x2     15.0  0.0  1.0  0.0  0.3 -0.166667
x1     75.0  1.0  0.0  0.0 -0.1  0.166667
最终的最优单纯形法是:
          b   x1   x2   x3   x4        x5
obj -5700.0  0.0  0.0  0.0 -2.0 -6.666667
x3    180.0  0.0  0.0  1.0 -2.4  1.000000
x2     15.0  0.0  1.0  0.0  0.3 -0.166667
x1     75.0  1.0  0.0  0.0 -0.1  0.166667
目标函数值是: 5700.0
最优决策变量是:
x1 = 75.0
x2 = 15.0


# 内点法

In [19]:
# 如何求解一阶导数和二阶导数

In [48]:
from sympy import diff, symbols, exp, log
# 定义变量
x1, x2, t = symbols('x1 x2 t')
# 定义目标函数
func = t * (-70*x1-30*x2) - log(-3*x1-9*x2+540) - log(-5*x1 - 5*x2+450) - log(-9*x1-3*x2+720) - log(-x1) - log(-x2)
# 求导
diff(func, x2, 1)
diff(func, x1, x1)
diff(func, x1, x2)
diff(func, x2, x1)
diff(func, x2, x2)

-30*t + 9/(-3*x1 - 9*x2 + 540) + 5/(-5*x1 - 5*x2 + 450) + 3/(-9*x1 - 3*x2 + 720) - 1/x2

9/(3*x1 + x2 - 240)**2 + (x1 + 3*x2 - 180)**(-2) + (x1 + x2 - 90)**(-2) + x1**(-2)

3/(3*x1 + x2 - 240)**2 + 3/(x1 + 3*x2 - 180)**2 + (x1 + x2 - 90)**(-2)

3/(3*x1 + x2 - 240)**2 + 3/(x1 + 3*x2 - 180)**2 + (x1 + x2 - 90)**(-2)

(3*x1 + x2 - 240)**(-2) + 9/(x1 + 3*x2 - 180)**2 + (x1 + x2 - 90)**(-2) + x2**(-2)

In [23]:
# 内点法代码实现

In [49]:
import numpy as np
import time

def gradient(x1, x2, t):
    """计算目标函数在x处的一阶导数（雅克比矩阵）"""
    j1 = -70 * t + 3 / (-3 * x1 - 9 * x2 + 540) + 5 / (-5 * x1 - 5 * x2 + 450) + 9 / (-9 * x1 - 3 * x2 + 720) - 1 / x1
    j2 = -30 * t + 9 / (-3 * x1 - 9 * x2 + 540) + 5 / (-5 * x1 - 5 * x2 + 450) + 3 / (-9 * x1 - 3 * x2 + 720) - 1 / x2
    return np.asmatrix([j1, j2]).T

def hessian(x1, x2):
    """计算目标函数在x处的二阶导数（海塞矩阵）"""
    x1, x2 = float(x1), float(x2)
    h11 = 9 / (3 * x1 + x2 - 240) ** 2 + (x1 + 3 * x2 - 180) ** (-2) + (x1 + x2 - 90) ** (-2) + x1 ** (-2)
    h12 = 3 / (3 * x1 + x2 - 240) ** 2 + 3 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2)
    h21 = 3 / (3 * x1 + x2 - 240) ** 2 + 3 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2)
    h22 = (3 * x1 + x2 - 240) ** (-2) + 9 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2) + x2 ** (-2)
    return np.asmatrix([[h11, h12], [h21, h22]])

def invertible(H):
    """求海塞矩阵的逆矩阵"""
    H_1 = np.linalg.inv(H)
    return H_1

def main():
    x = np.asmatrix(np.array([10, 10])).T  # x 是牛顿法的初始迭代值
    t = 0.00001  # t 是指示函数的中的t
    eps = 0.01  # 迭代停止的误差
    iter_cnt = 0  # 记录迭代的次数
    while iter_cnt < 20:
        iter_cnt += 1
        J = gradient(x[0, 0], x[1, 0], t)
        H = hessian(x[0, 0], x[1, 0])
        H_1 = np.linalg.inv(H)  # 还塞矩阵的逆
        x_new = x - H_1 * J  # 牛顿法公式
        error = np.linalg.norm(x_new - x)  # 求2范数，判断迭代效果
        print('迭代次数是：%d, x1=%.2f, x2=%.2f, 误差是：%.2f' % (iter_cnt, x_new[0, 0], x_new[1, 0], error))
        x = x_new
        if error < eps:
            break
        time.sleep(1)
    # 打印结果
    print("目标函数值是：%.2f" % float(70 * x[0, 0] + 30 * x[1, 0]))

In [50]:
main()

迭代次数是：1, x1=15.91, x2=15.34, 误差是：7.96
迭代次数是：2, x1=20.13, x2=18.19, 误差是：5.09
迭代次数是：3, x1=21.00, x2=18.33, 误差是：0.88
迭代次数是：4, x1=21.02, x2=18.32, 误差是：0.03
迭代次数是：5, x1=21.02, x2=18.32, 误差是：0.00
目标函数值是：2021.17


In [57]:
np.asmatrix(np.array([10, 10])).T

matrix([[10],
        [10]])